# Evristika

# Originalna

In [2]:
import numpy as np

def evaluate_state_amazons(board, amazons, arrows, player):
    """
    Проценка на состојбата за играта Амазон.
    
    Parameters:
    - board: numpy 2D array што ја претставува таблата.
    - amazons: Листа на координати [(x1, y1), (x2, y2), ...] за амазонките на дадениот играч.
    - arrows: Листа на координати [(x1, y1), ...] за стрелите на таблата.
    - player: 'W' или 'B' за бел или црн играч.
    
    Returns:
    - Вредност на евристиката за дадената состојба.
    """
    n = board.shape[0]  # Должина на страната на таблата.
    opponent = 'W' if player == 'B' else 'B'  # Одредување на противникот.
    
    def queen_moves(px, py):
        """Генерира сите можни полиња што кралицата може да ги достигне."""
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]  # Сите можни насоки на движење.
        moves = []  # Празна листа за чување на можните потези.
        for dx, dy in directions:  # За секоја насока...
            step = 1
            while True:
                nx, ny = px + step * dx, py + step * dy  # Калкулира нови координати.
                if 0 <= nx < n and 0 <= ny < n and board[nx, ny] == '.':  # Проверува дали е внатре во таблата и дали полето е празно.
                    moves.append((nx, ny))  # Додава ги координатите во листата.
                    step += 1
                else:
                    break  # Прекинува ако наиде на граница или зафатено поле.
        return moves
    
    # 1. **Територијална контрола (t1):**
    territory_score = 0  # Иницијација на резултат за територијална контрола.
    for x in range(n):  # Петља низ сите редови.
        for y in range(n):  # Петља низ сите колони.
            if board[x, y] == '.':  # Проверува дали полето е празно.
                player_dist = min(np.abs(x - ax) + np.abs(y - ay) for ax, ay in amazons)  # Калкулира растојание до најблиската амазонка.
                opponent_dist = min(np.abs(x - ax) + np.abs(y - ay) for ax, ay in arrows)  # Калкулира растојание до најблиската стрела.
                if player_dist < opponent_dist:
                    territory_score += 1  # Додава поен ако играчот е поблиску.
                elif player_dist > opponent_dist:
                    territory_score -= 1  # Одзема поен ако противникот е поблиску.
    
    # 2. **Мобилност на амазонките (m):**
    mobility_score = 0  # Иницијација на резултат за мобилност.
    for ax, ay in amazons:  # За секоја амазонка...
        mobility_score += len(queen_moves(ax, ay))  # Додава го бројот на можни потези.
    
    # 3. **Заканувачки потенцијал (h3 и h4):**
    threat_score = 0  # Иницијација на резултат за заканувачки потенцијал.
    for ax, ay in amazons:  # За секоја амазонка...
        reachable_squares = set(queen_moves(ax, ay))  # Генерира сите достапни полиња.
        threat_score += len(reachable_squares)  # Додава го бројот на достапни полиња.
    
    # 4. **Области на затворање:**
    enclosed_areas = 0  # Иницијација на резултат за затворање.
    for ax, ay in amazons:  # За секоја амазонка...
        if not queen_moves(ax, ay):  # Ако нема валидни потези...
            enclosed_areas -= 10  # Голема казна за затворени амазонки.
    
    # Вкупна евристичка вредност
    w1, w2, w3, w4 = 2.0, 3.0, 4.0, 1.0  # Тежини за секоја компонента.
    total_heuristic = w1 * territory_score + w2 * mobility_score + w3 * threat_score + w4 * enclosed_areas  # Калкулира вкупен резултат.
    
    return total_heuristic  # Враќа вкупен резултат.

In [3]:
import numpy as np

def test_evaluate_state_amazons():
    # Пример табла 10x10
    board = np.array([
        ['W', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', 'B', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ])

    # Амазонки и стрели
    amazons = [(2, 9)]  # Позиција на белата амазонка
    arrows = [(2, 4)]   # Стрела поставена од противникот
    player = 'B'

    # Повик на функцијата
    heuristic_value = evaluate_state_amazons(board, amazons, arrows, player)

    # Проверка на резултатите
    print(f"Евристичка вредност: {heuristic_value}")
    #assert isinstance(heuristic_value, (int, float)), "Резултатот треба да биде број."
    #assert heuristic_value != 0, "Евристиката треба да дава разлика помеѓу играчите."
test_evaluate_state_amazons()

Евристичка вредност: 113.0


# Izmeneta

In [6]:
import numpy as np

def assess_amazons_state(board, queens, fired_arrows, current_player):
    """
    Assess the state of the Amazons game for a specified player.

    Parameters:
    - board: numpy 2D array representing the game board.
    - queens: List of tuples [(x1, y1), (x2, y2), ...] representing the player's queen positions.
    - fired_arrows: List of tuples [(x1, y1), ...] representing the arrow positions on the board.
    - current_player: 'W' or 'B' for white or black player.

    Returns:
    - A heuristic value representing the state of the game.
    """
    board_size = board.shape[0]  # Length of one side of the square board.
    rival_player = 'W' if current_player == 'B' else 'B'  # Identify the rival player.

    def find_valid_moves(row, col):
        """Determine all valid moves for a queen from a given position."""
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]
        valid_moves = []
        for d_row, d_col in directions:
            distance = 1
            while True:
                new_row, new_col = row + distance * d_row, col + distance * d_col
                if 0 <= new_row < board_size and 0 <= new_col < board_size and board[new_row, new_col] == '.':
                    valid_moves.append((new_row, new_col))
                    distance += 1
                else:
                    break
        return valid_moves

    # 1. Calculate territorial influence.
    territorial_influence = 0
    for x in range(board_size):
        for y in range(board_size):
            if board[x, y] == '.':
                player_distance = min(abs(x - qx) + abs(y - qy) for qx, qy in queens)
                rival_distance = min(abs(x - ax) + abs(y - ay) for ax, ay in fired_arrows)
                if player_distance < rival_distance:
                    territorial_influence += 1
                elif player_distance > rival_distance:
                    territorial_influence -= 1

    # 2. Evaluate queen mobility.
    queen_mobility = 0
    for qx, qy in queens:
        queen_mobility += len(find_valid_moves(qx, qy))

    # 3. Assess potential reach (threat score).
    threat_score = 0
    for qx, qy in queens:
        reachable_positions = set(find_valid_moves(qx, qy))
        threat_score += len(reachable_positions)

    # 4. Penalize blocked queens.
    blocked_penalty = 0
    for qx, qy in queens:
        if not find_valid_moves(qx, qy):
            blocked_penalty -= 10

    # Aggregate heuristic components.
    weight_territory, weight_mobility, weight_threat, weight_blocked = 2.0, 3.0, 4.0, 1.0
    total_score = (
        weight_territory * territorial_influence +
        weight_mobility * queen_mobility +
        weight_threat * threat_score +
        weight_blocked * blocked_penalty
    )

    return total_score

In [7]:
import numpy as np

def test_evaluate_state_amazons():
    # Пример табла 10x10
    board = np.array([
        ['W', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', 'B', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ])

    # Амазонки и стрели
    amazons = [(2, 9)]  # Позиција на белата амазонка
    arrows = [(2, 4)]   # Стрела поставена од противникот
    player = 'B'

    # Повик на функцијата
    heuristic_value = assess_amazons_state(board, amazons, arrows, player)

    # Проверка на резултатите
    print(f"Евристичка вредност: {heuristic_value}")
    #assert isinstance(heuristic_value, (int, float)), "Резултатот треба да биде број."
    #assert heuristic_value != 0, "Евристиката треба да дава разлика помеѓу играчите."
test_evaluate_state_amazons()

Евристичка вредност: 113.0
